In [1]:
import pandas as pd
import requests
import base64
import json

In [2]:
pd.set_option('display.max_colwidth', 0)

In [3]:
pat = ""
authorization = str(base64.b64encode(bytes(':'+pat, 'ascii')), 'ascii')

headers = {
 'Accept': 'application/json',
 'Authorization': 'Basic '+authorization
}

In [4]:
df = pd.read_csv("us_cities.csv")

,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
0,1,AK,Alaska,Adak,Aleutians West,55.999722,-161.207778
1,2,AK,Alaska,Akiachak,Bethel,60.891854,-161.392330
2,3,AK,Alaska,Akiak,Bethel,60.890632,-161.199325
3,4,AK,Alaska,Akutan,Aleutians East,54.143012,-165.785368
4,5,AK,Alaska,Alakanuk,Kusilvak,62.746967,-164.602280


In [13]:
df[df['STATE_CODE']=='CA']

,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE
1684,1685,CA,California,Acampo,San Joaquin,38.200413,-121.218576
1685,1686,CA,California,Acton,Los Angeles,34.483541,-118.195929
1686,1687,CA,California,Adelanto,San Bernardino,34.584128,-117.424189
1687,1688,CA,California,Adin,Modoc,41.217510,-120.943193
1688,1689,CA,California,Agoura Hills,Los Angeles,34.157163,-118.757180
...,...,...,...,...,...,...,...
2921,2922,CA,California,Yuba City,Sutter,39.128619,-121.621599
2922,2923,CA,California,Yucaipa,San Bernardino,34.028197,-117.048925
2923,2924,CA,California,Yucca Valley,San Bernardino,34.155936,-116.431313
2924,2925,CA,California,Zamora,Yolo,38.799896,-121.906540


In [14]:
def form_url(row):
    
    long = row.LONGITUDE
    lat = row.LATITUDE
    
    url = "https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:{},{},1000&bias=proximity:{},{}&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f".format(long,lat,long,lat)
    return url


In [15]:
df['URL'] = df.apply (lambda row: form_url(row), axis=1)

'https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-161.207778,55.999722,1000&bias=proximity:-161.207778,55.999722&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f'

,ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE,URL
18874,18875,NY,New York,New York,New York,40.74838,-73.996705,"https://api.geoapify.com/v2/places?categories=tourism,entertainment,catering.restaurant&filter=circle:-73.996705,40.74838,1000&bias=proximity:-73.996705,40.74838&limit=500&apiKey=c8a825d7da834647b40102fcad6d5b3f"


In [28]:
top_cities = ["New York"]
#, "Los Angeles", "Las Vegas", "Orlando", "Miami", "San Francisco"

In [29]:
df_top = df[df['CITY'].isin(top_cities)]

In [30]:
df_top = df_top.reset_index()  # make sure indexes pair with number of rows

In [34]:
for index, row in df_top.iterrows():
    city = row["CITY"]
    state = row["STATE_CODE"]
    county = row["COUNTY"]
    url = row["URL"]
    filename = city+'-'+state+'.json'
    response = requests.get(url, headers=headers)

TypeError: 'Response' object is not subscriptable

In [48]:
if 'type' in response.content:
    del response.content['type']

TypeError: a bytes-like object is required, not 'str'

In [117]:
my_json = json.loads(response.content)
my_json.pop('type')
val = list(my_json.values())
my_json = val[0]
df = pd.json_normalize(my_json)

In [121]:
df

,type,properties.name,properties.housenumber,properties.street,properties.neighbourhood,properties.suburb,properties.county,properties.city,properties.state,properties.postcode,...,properties.datasource.raw.location,properties.datasource.raw.url,properties.datasource.raw.subject,properties.datasource.raw.operator:type,properties.datasource.raw.ref:nhrp,properties.datasource.raw.diet:meat,properties.datasource.raw.name:fr,properties.datasource.raw.name:fa,properties.datasource.raw.ref:nrhp,properties.datasource.raw.designation
0,Feature,Hoa Cafe,370,8th Avenue,Chelsea District,Manhattan,New York County,New York,New York,10001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feature,Banana Leaf,NaN,West 28th Street,Chelsea District,Manhattan,New York County,New York,New York,10001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Feature,Taco Bandito,325,8th Avenue,Chelsea District,Manhattan,New York County,New York,New York,10001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Feature,Bayard Rustin Residence,NaN,West 28th Street,Chelsea District,Manhattan,New York County,New York,New York,10001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Feature,Magnet Theater,254,West 29th Street,Chelsea District,Manhattan,New York County,New York,New York,10001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Feature,Flatiron Building,175,5th Avenue,Flatiron District,Manhattan,New York County,New York,New York,10010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ساختمان فلاتایرون,79001603.0,Office building
348,Feature,Kang Ho Dong Baekjeong,1,East 32nd Street,Midtown South,Manhattan,New York County,New York,New York,10016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,Feature,Golda Meir,NaN,West 39th Street,Garment District,Manhattan,New York County,New York,New York,10018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,Feature,Ladies Mile Historic District,NaN,West 20th Street,Chelsea,Manhattan,New York County,New York,New York,10011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'], port= 9042)
session = cluster.connect()
session.set_keyspace("Trippy")
query = "INSERT INTO data(date,time,open,high,low,last) VALUES (?,?,?,?,?,?)"
prepared = session.prepare(query)

ERROR: Could not find a version that satisfies the requirement pandra (from versions: none)
ERROR: No matching distribution found for pandra
Note: you may need to restart the kernel to use updated packages.
